In [62]:
!pip install tensorflow
import tensorflow as tf
import numpy as np
import pandas as pd


In [63]:
!pip install python-chess

In [64]:
import chess
from chess import pgn

In [65]:
from chess import pgn

def games_in(file):
    """
    Returns the list of games from a pgn file
    """
    games = open(file)
    game = pgn.read_game(games)
    games_list = []
    while game is not None:
        games_list.append(game)
        game = pgn.read_game(games)
    return games_list

In [66]:
from os import listdir
from os.path import isfile, join

def files_in(path):
    return [join(path,f) for f in listdir(path) if isfile(join(path, f))]

In [67]:
PIECES = chess.PIECE_TYPES

In [68]:
curpath = "pgn_games/Lichess Elite Database"

In [69]:
files = files_in(curpath)[: 20]

In [70]:
gameslist = []
for file in files:
    gameslist.extend(games_in(file))

In [93]:
columns = ['wMate', 'bMate', 'draw', 'turn', 
           'wP','wN','wB','wR','wQ','wK','bP','bN','bB','bR','bQ','bK',
           'hwP','hwN','hwB','hwR','hwQ','hwK','hbP','hbN','hbB','hbR','hbQ','hbK',
          'EVAL']

def features(board):
    """
    Returns a list of numerical values for features of a board. Subject to change.
    [ischeckmate[W/B], isDraw, turn, material[PNBRKQ][W/B] ,hangingMaterial[W/b]]
    """
    featuresList = []
    featuresList.extend(colorList(isCheckmate, board))
    featuresList.extend([isDraw(board)])
    featuresList.extend([currentTurn(board)])
    featuresList.extend(pieceColorList(numPieceOfColor, board))
    featuresList.extend(pieceColorList(numHangingPieceOfColor, board))
    return featuresList
    
def pieceColorList(fn, board):
    """
    Returns a list containing FN called on every piece type and color combo
    """
    a = [fn(board, piece, chess.WHITE) for piece in PIECES]
    a.extend([fn(board, piece, chess.BLACK)for piece in PIECES])
    return a
    
def colorList(fn, board):
    """
    Returns a list containing FN called on both color
    """
    a = [fn(board, chess.WHITE)]
    a.extend([fn(board, chess.BLACK)])
    return a
    
def isCheckmate(board, color):
    """
    Returns 1 if the current color is checkmating the other
    """
    if board.is_checkmate() and board.turn == (not color):
        return 1
    return 0

def isDraw(board):
    """
    Returns 1 iff the current position is a draw
    """
    if board.result() == '1/2-1/2': #might have to input some * in result
        return 1
    return 0

def currentTurn(board):
    if board.turn == chess.WHITE:
        return 1
    return 0

def numPieceOfColor(board, piece, color):
    """
    Returns the number of pieces of specific color
    """
    return len(board.pieces(piece, color))

def numHangingPieceOfColor(board, piece, color): 
    #Update function to still count pieces as hanging if num attacker pawns > defender pawns, repeat for each piece...
    """
    Returns the number of hanging pieces of a specific color 
    """
    our_pieces = board.pieces(piece, color)
    opp_color = not color
    hanging_pieces = 0
    for sq in our_pieces:
        unpinned_attackers_on_sq = sum([not board.is_pinned(not color, opp_att) for opp_att in board.attackers(opp_color, sq)])
        unpinned_defenders_on_sq = sum([not board.is_pinned(color, opp_att) for opp_att in board.attackers(color, sq)]) 
        if unpinned_attackers_on_sq > unpinned_defenders_on_sq:
            hanging_pieces += 1
    return hanging_pieces
        

In [94]:
testBoard = gameslist[0].board()
testBoard.push(chess.Move.from_uci('e2e4'))
testBoard.push(chess.Move.from_uci('e7e5'))
testBoard.push(chess.Move.from_uci('d1h5'))
testBoard.push(chess.Move.from_uci('g8f6'))
testBoard.push(chess.Move.from_uci('f1c4'))
testBoard.push(chess.Move.from_uci('b8c6'))
testBoard.push(chess.Move.from_uci('h5f7'))
print(testBoard)

print(features(testBoard))

r . b q k b . r
p p p p . Q p p
. . n . . n . .
. . . . p . . .
. . B . P . . .
. . . . . . . .
P P P P . P P P
R N B . K . N R
[1, 0, 0, 0, 8, 2, 2, 2, 1, 1, 7, 2, 2, 2, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [95]:
def length_of_game(game):
    return sum(1 for e in game.mainline_moves())

In [96]:
def add_game_to_dataset(array, game):
    board = game.board()
    length_game = length_of_game(game)
    current_move = 0
    result = game.headers["Result"]
    multiplier = 0
    if result == '1-0':
        multiplier = 1
    elif result == '0-1':
        multiplier = -1
    for move in game.mainline_moves():
        current_move += 1
        board.push(move)
        current_features = features(board)
        current_features.append(multiplier * current_move / length_game)
        array.append(current_features)

In [142]:
#IMPORT THE DATASET

dataset = []
modelgameslist = gameslist[:100]
for game in modelgameslist:
    add_game_to_dataset(dataset, game)
print(len(dataset))
print(sum([1 for i in dataset if i[28] > 0]))
data = pd.DataFrame(dataset, columns=columns)
x = data.iloc[:,:-1].values
y = data.iloc[:,-1].values


8360
3941


In [98]:
#SPLIT TRAIN AND TEST SET
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [100]:
#FEATURE SCALING
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [132]:
#BUILDING THE ANN
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=28, activation='relu'))
ann.add(tf.keras.layers.Dense(units=28, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='linear'))

In [133]:
#TRAINING THE ANN
ann.compile(optimizer='adam', loss='mean_squared_logarithmic_error')
ann.fit(x_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
209/209 [==============================] - 0s 759us/step - loss: 0.0642
Epoch 2/100
209/209 [==============================] - 0s 744us/step - loss: 0.0484
Epoch 3/100
209/209 [==============================] - 0s 701us/step - loss: 0.0443
Epoch 4/100
209/209 [==============================] - 0s 701us/step - loss: 0.0418
Epoch 5/100
209/209 [==============================] - 0s 740us/step - loss: 0.0403
Epoch 6/100
209/209 [==============================] - 0s 759us/step - loss: 0.0392
Epoch 7/100
209/209 [==============================] - 0s 735us/step - loss: 0.0378
Epoch 8/100
209/209 [==============================] - 0s 711us/step - loss: 0.0371
Epoch 9/100
209/209 [==============================] - 0s 725us/step - loss: 0.0364
Epoch 10/100
209/209 [==============================] - 0s 744us/step - loss: 0.0355
Epoch 11/100
209/209 [==============================] - 0s 721us/step - loss: 0.0350
Epoch 12/100
209/209 [==============================] - 0s 721us/step - lo

209/209 [==============================] - 0s 740us/step - loss: 0.0167
Epoch 98/100
209/209 [==============================] - 0s 697us/step - loss: 0.0166
Epoch 99/100
209/209 [==============================] - 0s 687us/step - loss: 0.0166
Epoch 100/100
209/209 [==============================] - 0s 764us/step - loss: 0.0163


In [139]:
testBoard = gameslist[0].board()
testBoard.push(chess.Move.from_uci('e2e4'))
testBoard.push(chess.Move.from_uci('e7e5'))
testBoard.push(chess.Move.from_uci('d1h5'))
testBoard.push(chess.Move.from_uci('g8f6'))
testBoard.push(chess.Move.from_uci('f1c4'))
testarray = [features(testBoard)]
sc.transform(testarray)
print(testarray)
ann.predict(testarray)


[[0, 0, 0, 0, 8, 2, 2, 2, 1, 1, 8, 2, 2, 2, 1, 1, 1, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0]]


array([[-1.0067983]], dtype=float32)